In [1]:
# Kaggle_Data_Extraction
# This class containes the code for
#    1. Reading the cricket player/team data from external csv files
#    2. Cleaning of the data read from exteral source
#    3. Exporting the cleaned data into a csv file
#  Detailed description availble through inline comments.

import pandas as pd
import numpy as np
import matplotlib as plt
import json
import re
from re import sub
from pandas.io.json import json_normalize
pd.options.mode.chained_assignment = None 



# Team Ranking
# Team ranking data read from csv file 'ODIs - ICC Rankings.csv' and being stored under dataframe 'teamranking_df'
teamranking_df= pd.read_csv('ODIs - ICC Rankings.csv')
# Removal of column 'Weighted Matches as it not required for conceptual data model'
del teamranking_df['Weighted Matches']
# Removal of ',' and conversion of the datatype to integer for 'Points' column
teamranking_df['Points'] = teamranking_df['Points'].str.replace(",","")
teamranking_df['Points'] = teamranking_df['Points'].astype(int)
# Conversion of lower case and removal of whitespace for 'Team_ID' column
teamranking_df['Team_ID'] = teamranking_df['Country'].str.lower()
teamranking_df['Team_ID'] = teamranking_df['Team_ID'].replace(' ', '')
# Renaming columns 'Country' and 'Pos' to 'Team_Name' and 'Rank'
teamranking_df = teamranking_df.rename(columns=({'Country':'Team_Name',
                                         'Pos':'Rank'}))



# Team Batting
# Team batting data read from csv file 'ODIs - Teams Batting.csv' and being stored under dataframe 'teambatting_df'
teambatting_df= pd.read_csv('ODIs - Teams Batting.csv')
# Removal of column 'Win/Loss Ratio' as it not required for conceptual data model'
del teambatting_df['Win/Loss Ratio']
# Conversion of lower case and removal of whitespace for 'Team_ID' column
teambatting_df['Team_ID'] = teambatting_df['Country'].str.lower()
teambatting_df['Team_ID'] = teambatting_df['Team_ID'].replace(' ', '')
# Renaming columns 'Country' to 'Team_Name'
teambatting_df = teambatting_df.rename(columns=({'Country':'Team_Name'}))
# Replacing blank space with '_' in all columns of teambatting_df
teambatting_df.columns = teambatting_df.columns.str.replace(' ', '_')



teamranking_df, teambatting_df = [d.reset_index(drop=True) for d in (teamranking_df, teambatting_df)]
#Creation of new dataframe teamgeneral_df by merging teamranking_df and teambatting_df with keys Team_ID and Team_Name
teamgeneral_df = (pd.merge(teamranking_df, teambatting_df, on=['Team_ID', 'Team_Name']).reindex(columns=['Team_ID', 'Team_Name', 'Rank', 'Points',
          'Rating','Team_Matches_Played','Matches_Won','Matches_Lost','Matches_Tied','Matches_With_No_Result']))
# Removal of columns as they are not required for conceptual data model
del teambatting_df['Team_Matches_Played']
del teambatting_df['Matches_Won']
del teambatting_df['Matches_Lost']
del teambatting_df['Matches_Tied']
del teambatting_df['Matches_With_No_Result']



#Team Bowling
# Team bowling data read from csv file 'ODIs - Teams Bowling.csv' and being stored under dataframe 'teambowling_df'
teambowling_df= pd.read_csv('ODIs - Teams Bowling.csv')
# Conversion of lower case and removal of whitespace for 'Team_ID' column
teambowling_df['Team_ID'] = teambowling_df['Country'].str.lower()
teambowling_df['Team_ID'] = teambowling_df['Team_ID'].str.replace(' ', '')
# Renaming columns 'Country' to 'Team_Name'
teambowling_df = teambowling_df.rename(columns=({'Country':'Team_Name'}))
# Replacing blank space with '_' in all columns of teambatting_df
teambowling_df.columns = teambowling_df.columns.str.replace(' ', '_')



#Player Batting
# Player batting data read from csv file 'ODIs - Batting.csv' and being stored under dataframe 'player_batting_df'
player_batting_df = pd.read_csv('ODIs - Batting.csv')
# Removal of bracketed words in Player column
player_batting_df['Player'] = player_batting_df['Player'].str.replace(r"\(.*\)","")
# Conversion of lower case and removal of whitespace for 'Player_ID' column
player_batting_df['Player_ID'] = player_batting_df['Player'].str.lower()
player_batting_df['Player_ID'] = player_batting_df['Player_ID'].str.replace(' ', '')
# Conversion of lower case and removal of whitespace for 'Team_ID' column
player_batting_df['Team_ID'] = player_batting_df['Country'].str.lower()
player_batting_df['Team_ID'] = player_batting_df['Team_ID'].str.replace(' ', '')
# Replacing blank space with '_' in all columns of player_batting_df
player_batting_df.columns = player_batting_df.columns.str.replace(' ', '_')
# Renaming columns 'Player' to 'Player_Name'
player_batting_df = player_batting_df.rename(columns=({'Player':'Player_Name'}))


# Creation of new dataframe player_general_df by copying selecive columns from 'player_batting_df'
player_general_df = player_batting_df[['Player_ID', 'Player_Name', 'Career_Start', 'Career_End', 'Matches_Played','Team_ID']].copy()



# Removal of columns as they are not required for conceptual data model
player_batting_df.drop(['Career_Span', 'Career_Start','Career_End','Matches_Played','Highest_Innings_Score',
                       'Player_Count','Country','5000+_Runs_Scored','40+_Batting_Avg','90+_Batting_Strike_Rate'], axis=1, inplace=True)
# Renaming columns 'Highest_Innings_Score_Num' and 'Batting_Avg'
player_batting_df = player_batting_df.rename(columns=({'Highest_Innings_Score_Num':'Highest_Innings_Score',
                                                       'Batting_Avg':'Batting_Average'}))
# Dropping 'Team_ID column' as it is already presnet in 'player_general_df', which is the main dataframe for player 
#and connected to player_batting_df
player_batting_df.drop(['Team_ID'], axis=1, inplace=True)


#Player Bowling
# Player bowling data read from csv file 'ODIs - Bowling.csv' and being stored under dataframe 'player_bowling_df'
player_bowling_df = pd.read_csv('ODIs - Bowling.csv')
# Removal of bracketed words in Player column
player_bowling_df['Player'] = player_bowling_df['Player'].str.replace(r"\(.*\)","")
# Conversion of lower case and removal of whitespace for 'Player_ID' column
player_bowling_df['Player_ID'] = player_bowling_df['Player'].str.lower()
player_bowling_df['Player_ID'] = player_bowling_df['Player_ID'].str.replace(' ', '')
# Replacing blank space with '_' in all columns of player_bowling_df
player_bowling_df.columns = player_bowling_df.columns.str.replace(' ', '_')
# Renaming column 'Player'
player_bowling_df = player_bowling_df.rename(columns=({'Player':'Player_Name'}))
# Dropping columns as they are not required for conceptual data model 
player_bowling_df.drop(['Best_Bowling_In_An_Innings', '200+_Wickets_Taken','<35.00_Bowling_Avg','<4.00_Economy_Rate','<40.00_Bowling_Strike_Rate'], axis=1, inplace=True)
# Renaming column 'Bowling_Avg'
player_bowling_df = player_bowling_df.rename(columns=({'Bowling_Avg':'Bowling_Average'}))



#Player Fielding
# Player Fielding data read from csv file 'ODIs - Fielding.csv' and being stored under dataframe 'player_fielding_df'
player_fielding_df = pd.read_csv('ODIs - Fielding.csv')
# Removal of bracketed words in Player column
player_fielding_df['Player'] = player_fielding_df['Player'].str.replace(r"\(.*\)","")
# Conversion of lower case and removal of whitespace for 'Player_ID' column
player_fielding_df['Player_ID'] = player_fielding_df['Player'].str.lower()
player_fielding_df['Player_ID'] = player_fielding_df['Player_ID'].str.replace(' ', '')
# Replacing blank space with '_' in all columns of player_fielding_df
player_fielding_df.columns = player_fielding_df.columns.str.replace(' ', '_')
# Renaming column 'Player'
player_fielding_df = player_fielding_df.rename(columns=({'Player':'Player_Name'}))
# Dropping columns as they are not required for conceptual data model 
player_fielding_df.drop(['Catches_As_A_Keeper', 'Catches_As_A_Fielder','Max_Dismissals_In_An_Innings'], axis=1, inplace=True)


# Creating player_general_2018_100_df, which contains only records of players whose 'CareerEnd' is 2018(latest/current players) 
# and matched palyed is more than 100, so that we get popular and well known players
player_general_2018_100_df = player_general_df.loc[(player_general_df['Career_End'] == 2018) & (player_general_df['Matches_Played'] >= 100)]
# Creation of a new list from 'Player_ID' column of player_general_2018_100_df
player_general_2018_100_key_list = player_general_2018_100_df['Player_ID']



#Creating respective filtered dataframes for batting, bowling and fielding using the key column 'Player_ID' which has the 
# filtered values of career end = 2018 and matches played is greater tan or equal to 100
player_batting_2018_100_df = player_batting_df.loc[player_batting_df['Player_ID'].isin(player_general_2018_100_key_list)]
player_bowling_2018_100_df = player_bowling_df.loc[player_bowling_df['Player_ID'].isin(player_general_2018_100_key_list)]
player_fielding_2018_100_df = player_fielding_df.loc[player_fielding_df['Player_ID'].isin(player_general_2018_100_key_list)]


#Removal of duplicates in the below dataframes with regard to 'Player_ID'(which would later serve as primary key in related table)
# Only the first record is kept, while other records with same value are removed
player_batting_2018_100_df.drop_duplicates(subset=['Player_ID'], keep='first', inplace= True)
player_bowling_2018_100_df.drop_duplicates(subset=['Player_ID'], keep='first', inplace= True)
player_fielding_2018_100_df.drop_duplicates(subset=['Player_ID'], keep='first', inplace= True)
player_general_2018_100_df.drop_duplicates(subset=['Player_ID'], keep='first', inplace= True)


# Dropping 'Player_Name column' as it is already presnet in 'player_general_2018_100_dff', which is the main dataframe for player 
#and connected to player_batting/bowling/fielding_2018_100_dfs
player_batting_2018_100_df.drop(['Player_Name'], axis=1, inplace=True)
player_bowling_2018_100_df.drop(['Player_Name'], axis=1, inplace=True)
player_fielding_2018_100_df.drop(['Player_Name'], axis=1, inplace=True)


# Dropping 'Team_Name column' as it is already presnet in 'player_general_dff', which is the main dataframe for team 
#and connected to team_batting/bowling_dfs
teambatting_df.drop(['Team_Name'], axis=1, inplace=True)
teambowling_df.drop(['Team_Name'], axis=1, inplace=True)


# Replacing vlaues with '-' as 0 in the below columns
player_bowling_2018_100_df.loc[player_bowling_2018_100_df['Innings_Bowled_In'] == '-', 'Innings_Bowled_In'] = 0
player_bowling_2018_100_df = player_bowling_2018_100_df.rename(columns=({'Bowling Average':'Bowling_Average'}))
player_bowling_2018_100_df.loc[player_bowling_2018_100_df['Balls_Bowled'] == '-', 'Balls_Bowled'] = 0
player_bowling_2018_100_df.loc[player_bowling_2018_100_df['Runs_Conceded'] == '-', 'Runs_Conceded'] = 0
player_bowling_2018_100_df.loc[player_bowling_2018_100_df['Wickets_Taken'] == '-', 'Wickets_Taken'] = 0
player_bowling_2018_100_df.loc[player_bowling_2018_100_df['Economy_Rate'] == '-', 'Economy_Rate'] = 0
player_bowling_2018_100_df.loc[player_bowling_2018_100_df['Bowling_Strike_Rate'] == '-', 'Bowling_Strike_Rate'] = 0
player_bowling_2018_100_df.loc[player_bowling_2018_100_df['Four_Wickets_In_An_Innings'] == '-', 'Four_Wickets_In_An_Innings'] = 0
player_bowling_2018_100_df.loc[player_bowling_2018_100_df['Five_Wickets_In_An_Innings'] == '-', 'Five_Wickets_In_An_Innings'] = 0
player_bowling_2018_100_df.loc[player_bowling_2018_100_df['Bowling_Average'] == '-', 'Bowling_Average'] = 0


# Removal of ',' symbol, conversion to float, removal of NAN values and conversion to Integer datatye in 'Balls_Bowled' column
player_bowling_2018_100_df['Balls_Bowled'] = player_bowling_2018_100_df['Balls_Bowled'].str.replace(",","").astype(float)
player_bowling_2018_100_df['Balls_Bowled'].fillna(0, inplace=True)
player_bowling_2018_100_df['Balls_Bowled'] = player_bowling_2018_100_df['Balls_Bowled'].astype(int)['Balls_Bowled'] = player_bowling_2018_100_df['Balls_Bowled'].astype(int)


# Conversion of relevant datatype for the below numerical columns, which were of 'Object' type earlier in player_bowling_2018_100_df
player_bowling_2018_100_df['Innings_Bowled_In'] = player_bowling_2018_100_df['Innings_Bowled_In'].astype(int)
player_bowling_2018_100_df['Runs_Conceded'] = player_bowling_2018_100_df['Runs_Conceded'].astype(int)
player_bowling_2018_100_df['Wickets_Taken'] = player_bowling_2018_100_df['Wickets_Taken'].astype(int)
player_bowling_2018_100_df['Bowling_Average'] = player_bowling_2018_100_df['Bowling_Average'].astype(float)
player_bowling_2018_100_df['Economy_Rate'] = player_bowling_2018_100_df['Economy_Rate'].astype(float)
player_bowling_2018_100_df['Bowling_Strike_Rate'] = player_bowling_2018_100_df['Bowling_Strike_Rate'].astype(float)
player_bowling_2018_100_df['Four_Wickets_In_An_Innings'] = player_bowling_2018_100_df['Four_Wickets_In_An_Innings'].astype(int)
player_bowling_2018_100_df['Five_Wickets_In_An_Innings'] = player_bowling_2018_100_df['Five_Wickets_In_An_Innings'].astype(int)


# Conversion of relevant datatype for the below numerical columns, which were of 'Object' type earlier in player_batting_2018_100_df
player_batting_2018_100_df['Innings_Batted'] = player_batting_2018_100_df['Innings_Batted'].astype(int)
player_batting_2018_100_df['Not_Outs'] = player_batting_2018_100_df['Not_Outs'].astype(int)
player_batting_2018_100_df['Runs_Scored'] = player_batting_2018_100_df['Runs_Scored'].astype(int)
player_batting_2018_100_df['Highest_Innings_Score'] = player_batting_2018_100_df['Highest_Innings_Score'].astype(int)
player_batting_2018_100_df['Balls_Faced'] = player_batting_2018_100_df['Balls_Faced'].astype(int)
player_batting_2018_100_df['Batting_Average'] = player_batting_2018_100_df['Batting_Average'].astype(float)
player_batting_2018_100_df['Batting_Strike_Rate'] = player_batting_2018_100_df['Batting_Strike_Rate'].astype(float)
player_batting_2018_100_df['Hundreds_Scored'] = player_batting_2018_100_df['Hundreds_Scored'].astype(int)
player_batting_2018_100_df['Scores_Of_Fifty_Or_More'] = player_batting_2018_100_df['Scores_Of_Fifty_Or_More'].astype(int)
player_batting_2018_100_df['Ducks_Scored'] = player_batting_2018_100_df['Ducks_Scored'].astype(int)


player_general_2018_100_df.to_csv("Kaggle_Cleaned_Player.csv", index=False)
player_batting_2018_100_df.to_csv("Kaggle_Cleaned_PlayerBattingStatistics.csv", index=False)
player_bowling_2018_100_df.to_csv("Kaggle_Cleaned_PlayerBowlingStatistics.csv", index=False)
player_fielding_2018_100_df.to_csv("Kaggle_Cleaned_PlayerFieldingStatistics.csv", index=False)


teamgeneral_df.to_csv("Kaggle_Cleaned_Team.csv", index=False)
teambatting_df.to_csv("Kaggle_Cleaned_TeamBattingStatistics.csv", index=False)
teambowling_df.to_csv("Kaggle_Cleaned_TeamBowlingStatistics.csv", index=False)

""""
CONCLUSIONS:

This code reads data taken from an external source 'Kaggle', performs data cleaning, data reformat, stores it in a dataframe and 
finally exports it to a CSV file

CONTRIBUTIONS:

MONISH  HIRISAVE RAGHU - 75%
RAJENDRA KUMAR RAJKUMAR - 25%


CITATIONS:

1. https://www.geeksforgeeks.org
2. https://github.com/nikbearbrown/INFO_6210
3. stackoverflow
4. Tutorialspoint

In built functions and methods required for data cleaning, formatting options were referred from above mentioned sites

Percentage of code written - 90%
Percentage of code referred from above mentioned scources - 10%

LICENSE:

Copyright <2019> <RAJENDRA KUMAR RAJKUMAR, MONISH  HIRISAVE RAGHU>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


"""

'"\nCONCLUSIONS:\n\nThis code reads data taken from an external source \'Kaggle\', performs data cleaning, data reformat, stores it in a dataframe and \nfinally exports it to a CSV file\n\nCONTRIBUTIONS:\n\nMONISH  HIRISAVE RAGHU - 75%\nRAJENDRA KUMAR RAJKUMAR - 25%\n\n\nCITATIONS:\n\n1. https://www.geeksforgeeks.org\n2. https://github.com/nikbearbrown/INFO_6210\n3. stackoverflow\n4. Tutorialspoint\n\nIn built functions and methods required for data cleaning, formatting options were referred from above mentioned sites\n\nPercentage of code written - 90%\nPercentage of code referred from above mentioned scources - 10%\n\nLICENSE:\n\nCopyright <2019> <RAJENDRA KUMAR RAJKUMAR, MONISH  HIRISAVE RAGHU>\n\nPermission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicen